In [1]:
import numpy as np
user_matrice = np.load('user_matrice_filtered.npy') # 104 days x 57000 users x 3 weightings
user_stock_matrice = np.load('user_stock_matrice_filtered.npy') # 104 days x 78 stocks x 57000 users x 3 weightings
user_predictions = np.load('user_predictions.npy') # 104 days x 57000 users x 78 stocks
close_open_matrice = np.load('close_open_matrice.npy') # 104 x 78
print('done loading')


done loading


In [109]:
from scipy import sparse
sA = sparse.csr_matrix(user_matrice[0][0])

np.save('test1.npy', sA)
np.save('test2.npy', user_matrice[0][0])


result = []
for i in range(104):
    sA = sparse.csr_matrix(user_predictions[i])
    result.append(sA)

np.save('user_predictions_sparse.npy', result)

In [2]:
weightings=np.array([1,1,1])
user_stock_weighted_bull = np.dot(user_stock_matrice[0], weightings) # 104 x 57000
user_stock_weighted_bear = np.dot(user_stock_matrice[1], weightings) # 104 x 57000


In [4]:
masked_bull_predictions = np.array(user_predictions, copy=True)
masked_bull_predictions[masked_bull_predictions<0] = 0
count_total_bull = np.count_nonzero(masked_bull_predictions, axis=1) # counts per day per stock
standardize_bull_prediction = masked_bull_predictions / count_total_bull[:,None]
standardize_bull_prediction[np.isnan(standardize_bull_prediction)] = 0

masked_bear_predictions = np.array(user_predictions, copy=True)
masked_bear_predictions[masked_bear_predictions>0] = 0
count_total_bear = np.count_nonzero(masked_bear_predictions, axis=1) # counts per day per stock
standardize_bear_prediction = masked_bear_predictions / count_total_bear[:,None]
standardize_bear_prediction[np.isnan(standardize_bear_prediction)] = 0

In [5]:
diagonal_weights_bull_stock = np.einsum('ijk,ikj->ij', user_stock_weighted_bull, standardize_bull_prediction, optimize=True) # 104 x 78
diagonal_weights_bear_stock = np.einsum('ijk,ikj->ij', user_stock_weighted_bear, standardize_bear_prediction, optimize=True) # 104 x 78

In [58]:
mask = diagonal_weights_bull_stock < abs(diagonal_weights_bear_stock)
index = 10
print(diagonal_weights_bull_stock[index])
print(diagonal_weights_bear_stock[index])
mask[index]

[1.04687799 1.18435081 0.         1.23221204 0.         1.1511525
 0.         1.24746159 0.71872587 0.         1.05157076 1.24090793
 0.         0.         0.         0.         0.         1.03606915
 1.1012869  0.         0.         0.         0.67959675 0.
 0.         0.         0.         0.         0.         0.99886037
 0.         0.96815518 1.0719881  1.00620285 0.69861657 1.19039692
 0.         0.         0.         0.         1.0054635  0.
 0.         0.64860393 0.         1.30100037 0.         0.        ]
[-0.52438486 -0.94444623  0.         -0.76440454  0.         -0.82044122
  0.         -1.02134225 -0.7375802   0.         -0.85981158 -0.67762816
  0.          0.          0.          0.          0.         -0.51763163
 -1.16853184  0.          0.          0.         -0.30498541  0.
  0.          0.          0.          0.          0.         -0.66356984
  0.         -0.6656116  -0.50426296 -0.66664704 -0.47374777 -0.79649668
  0.          0.          0.          0.         -

array([False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False])

In [53]:
copy_bull = np.array(diagonal_weights_bull_stock, copy=True)
copy_bear = np.array(diagonal_weights_bear_stock, copy=True)

for i in range(len(copy_bull)):
    for j in range(len(copy_bull[i])):
        if (abs(copy_bear[i][j]) > copy_bull[i][j]):
            temp = copy_bull[i][j]
            copy_bull[i][j] = copy_bear[i][j]
            copy_bear[i][j] = temp
        else:
            copy_bear[i][j] = abs(copy_bear[i][j])
res = copy_bull / copy_bear
res[res > 0] -= 1
res[res < 0] += 1
res[np.isnan(res)] = 0
res[np.isinf(res)] = 1


In [57]:
res[10]

array([ 0.9963925 ,  0.25401613,  0.        ,  0.61198943,  0.        ,
        0.40308955,  0.        ,  0.22139429, -0.02623299,  0.        ,
        0.22302466,  0.831252  ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.00155686, -0.06106033,  0.        ,
        0.        ,  0.        ,  1.22829266,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.50528294,
        0.        ,  0.45453471,  1.12585136,  0.50934872,  0.47465933,
        0.49454096,  0.        ,  0.        ,  0.        ,  0.        ,
        0.34138222,  0.        ,  0.        , -0.71327521,  0.        ,
       11.02874903,  0.        ,  0.        ])

In [7]:
# Return list of valid trading days from date on
def findTradingDays(date, upToDate):
    delta = datetime.timedelta(1)
    trading_days = set(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-16', '2018-01-17', '2018-01-18', '2018-01-19', '2018-01-22', '2018-01-23', '2018-01-24', '2018-01-25', '2018-01-26', '2018-01-29', '2018-01-30', '2018-01-31', '2018-02-01', '2018-02-02', '2018-02-05', '2018-02-06', '2018-02-07', '2018-02-08', '2018-02-09', '2018-02-12', '2018-02-13', '2018-02-14', '2018-02-15', '2018-02-16', '2018-02-20', '2018-02-21', '2018-02-22', '2018-02-23', '2018-02-26', '2018-02-27', '2018-02-28', '2018-03-01', '2018-03-02', '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08', '2018-03-09', '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', '2018-03-16', '2018-03-19', '2018-03-20', '2018-03-21', '2018-03-22', '2018-03-23', '2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29', '2018-04-02', '2018-04-03', '2018-04-04', '2018-04-05', '2018-04-06', '2018-04-09', '2018-04-10', '2018-04-11', '2018-04-12', '2018-04-13', '2018-04-16', '2018-04-17', '2018-04-18', '2018-04-19', '2018-04-20', '2018-04-23', '2018-04-24', '2018-04-25', '2018-04-26', '2018-04-27', '2018-04-30', '2018-05-01', '2018-05-02', '2018-05-03', '2018-05-04', '2018-05-07', '2018-05-08', '2018-05-09', '2018-05-10', '2018-05-11', '2018-05-14', '2018-05-15', '2018-05-16', '2018-05-17', '2018-05-18', '2018-05-21', '2018-05-22', '2018-05-23', '2018-05-24', '2018-05-25', '2018-05-29', '2018-05-30', '2018-05-31', '2018-06-01', '2018-06-04', '2018-06-05', '2018-06-06', '2018-06-07', '2018-06-08', '2018-06-11', '2018-06-12', '2018-06-13', '2018-06-14', '2018-06-15', '2018-06-18', '2018-06-19', '2018-06-20', '2018-06-21', '2018-06-22', '2018-06-25', '2018-06-26', '2018-06-27', '2018-06-28', '2018-06-29', '2018-07-02', '2018-07-03', '2018-07-05', '2018-07-06', '2018-07-09', '2018-07-10', '2018-07-11', '2018-07-12', '2018-07-13', '2018-07-16', '2018-07-17', '2018-07-18', '2018-07-19', '2018-07-20', '2018-07-23', '2018-07-24', '2018-07-25', '2018-07-26', '2018-07-27', '2018-07-30', '2018-07-31', '2018-08-01', '2018-08-02', '2018-08-03', '2018-08-06', '2018-08-07', '2018-08-08', '2018-08-09', '2018-08-10', '2018-08-13', '2018-08-14', '2018-08-15', '2018-08-16', '2018-08-17', '2018-08-20', '2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24', '2018-08-27', '2018-08-28', '2018-08-29', '2018-08-30', '2018-08-31', '2018-09-04', '2018-09-05', '2018-09-06', '2018-09-07', '2018-09-10', '2018-09-11', '2018-09-12', '2018-09-13', '2018-09-14', '2018-09-17', '2018-09-18', '2018-09-19', '2018-09-20', '2018-09-21', '2018-09-24', '2018-09-25', '2018-09-26', '2018-09-27', '2018-09-28', '2018-10-01', '2018-10-02', '2018-10-03', '2018-10-04', '2018-10-05', '2018-10-08', '2018-10-09', '2018-10-10', '2018-10-11', '2018-10-12', '2018-10-15', '2018-10-16', '2018-10-17', '2018-10-18', '2018-10-19', '2018-10-22', '2018-10-23', '2018-10-24', '2018-10-25', '2018-10-26', '2018-10-29', '2018-10-30', '2018-10-31', '2018-11-01', '2018-11-02', '2018-11-05', '2018-11-06', '2018-11-07', '2018-11-08', '2018-11-09', '2018-11-12', '2018-11-13', '2018-11-14', '2018-11-15', '2018-11-16', '2018-11-19', '2018-11-20', '2018-11-21', '2018-11-23', '2018-11-26', '2018-11-27', '2018-11-28', '2018-11-29', '2018-11-30', '2018-12-03', '2018-12-04', '2018-12-06', '2018-12-07', '2018-12-10', '2018-12-11', '2018-12-12', '2018-12-13', '2018-12-14', '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20', '2018-12-21', '2018-12-24', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-31', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31', '2019-02-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-11', '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-03-01', '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-11', '2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15', '2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21', '2019-03-22', '2019-03-25', '2019-03-26', '2019-03-27', '2019-03-28', '2019-03-29', '2019-04-01', '2019-04-02', '2019-04-03', '2019-04-04', '2019-04-05', '2019-04-08', '2019-04-09', '2019-04-10', '2019-04-11', '2019-04-12', '2019-04-15', '2019-04-16', '2019-04-17', '2019-04-18', '2019-04-22', '2019-04-23', '2019-04-24', '2019-04-25', '2019-04-26', '2019-04-29', '2019-04-30', '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-06', '2019-05-07', '2019-05-08', '2019-05-09', '2019-05-10', '2019-05-13', '2019-05-14', '2019-05-15', '2019-05-16', '2019-05-17', '2019-05-20', '2019-05-21', '2019-05-22', '2019-05-23', '2019-05-24', '2019-05-28', '2019-05-29', '2019-05-30', '2019-05-31', '2019-06-03', '2019-06-04', '2019-06-05', '2019-06-06', '2019-06-07', '2019-06-10', '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-17', '2019-06-18', '2019-06-19', '2019-06-20', '2019-06-21', '2019-06-24', '2019-06-25', '2019-06-26', '2019-06-27', '2019-06-28', '2019-07-01', '2019-07-02', '2019-07-03', '2019-07-05', '2019-07-08', '2019-07-09', '2019-07-10', '2019-07-11', '2019-07-12', '2019-07-15', '2019-07-16', '2019-07-17', '2019-07-18', '2019-07-19', '2019-07-22', '2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-29', '2019-07-30', '2019-07-31', '2019-08-01', '2019-08-02', '2019-08-05', '2019-08-06', '2019-08-07', '2019-08-08', 
        '2019-08-09', '2019-08-12', '2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16', '2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22', '2019-08-23', '2019-08-26', '2019-08-27', '2019-08-28', '2019-08-29', '2019-08-30', '2019-09-03', '2019-09-04', '2019-09-05', '2019-09-06', '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12', '2019-09-13', '2019-09-16', '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20', '2019-09-23', '2019-09-24', '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-30', '2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04', '2019-10-07', '2019-10-08', '2019-10-09', '2019-10-10', '2019-10-11', '2019-10-14', '2019-10-15', '2019-10-16', '2019-10-17', '2019-10-18', '2019-10-21', '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25', '2019-10-28', '2019-10-29', '2019-10-30', '2019-10-31', '2019-11-01', '2019-11-04', '2019-11-05', '2019-11-06', '2019-11-07', '2019-11-08', '2019-11-11', '2019-11-12', '2019-11-13', '2019-11-14', '2019-11-15', '2019-11-18', '2019-11-19', '2019-11-20', '2019-11-21', '2019-11-22', '2019-11-25', '2019-11-26', '2019-11-27', '2019-11-29', '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05', '2019-12-06', '2019-12-09', '2019-12-10', '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27', '2019-12-30', '2019-12-31', '2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-04', '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-08', '2020-06-09', '2020-06-10', '2020-06-11', '2020-06-12', '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18', '2020-06-19', '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26', '2020-06-29', '2020-06-30', '2020-07-01', '2020-07-02', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10', '2020-07-13', '2020-07-14', '2020-07-15', '2020-07-16', '2020-07-17', '2020-07-20', '2020-07-21', '2020-07-22', '2020-07-23', '2020-07-24', '2020-07-27', '2020-07-28', '2020-07-29', '2020-07-30', '2020-07-31', '2020-08-03', '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13', '2020-08-14', '2020-08-17', '2020-08-18', '2020-08-19', '2020-08-20', '2020-08-21', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-31', '2020-09-01', '2020-09-02', '2020-09-03', '2020-09-04', '2020-09-08', '2020-09-09', '2020-09-10', '2020-09-11', '2020-09-14', '2020-09-15', '2020-09-16', '2020-09-17', '2020-09-18', '2020-09-21', '2020-09-22', '2020-09-23', '2020-09-24', '2020-09-25', '2020-09-28', '2020-09-29', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-08', '2020-10-09', '2020-10-12', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30', '2020-11-02', '2020-11-03', '2020-11-04', '2020-11-05', '2020-11-06', '2020-11-09', '2020-11-10', '2020-11-11', '2020-11-12', '2020-11-13', '2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19', '2020-11-20', '2020-11-23', '2020-11-24', '2020-11-25', '2020-11-27', '2020-11-30', '2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04', '2020-12-07', '2020-12-08', '2020-12-09', '2020-12-10', '2020-12-11', '2020-12-14', '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18', '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-15', '2021-03-16', '2021-03-17', 
        '2021-03-18', '2021-03-19', '2021-03-22', '2021-03-23', '2021-03-24', '2021-03-25', '2021-03-26', '2021-03-29', '2021-03-30', '2021-03-31', '2021-04-01', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08', '2021-04-09', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15', '2021-04-16', '2021-04-19', '2021-04-20', '2021-04-21', '2021-04-22', '2021-04-23', '2021-04-26', '2021-04-27', '2021-04-28', '2021-04-29', '2021-04-30', '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24', '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28', '2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04', '2021-06-07', '2021-06-08', '2021-06-09', '2021-06-10', '2021-06-11', '2021-06-14', '2021-06-15', '2021-06-16', '2021-06-17', '2021-06-18', '2021-06-21', '2021-06-22', '2021-06-23', '2021-06-24', '2021-06-25', '2021-06-28', '2021-06-29', '2021-06-30', '2021-07-01', '2021-07-02', '2021-07-06', '2021-07-07', '2021-07-08', '2021-07-09', '2021-07-12', '2021-07-13', '2021-07-14', '2021-07-15', '2021-07-16', '2021-07-19', '2021-07-20', '2021-07-21', '2021-07-22', '2021-07-23', '2021-07-26', '2021-07-27', '2021-07-28', '2021-07-29', '2021-07-30', '2021-08-02', '2021-08-03', '2021-08-04', '2021-08-05', '2021-08-06', '2021-08-09', '2021-08-10', '2021-08-11', '2021-08-12', '2021-08-13', '2021-08-16', '2021-08-17', '2021-08-18', '2021-08-19', '2021-08-20', '2021-08-23', '2021-08-24', '2021-08-25', '2021-08-26', '2021-08-27', '2021-08-30', '2021-08-31', '2021-09-01', '2021-09-02', '2021-09-03', '2021-09-07', '2021-09-08', '2021-09-09', '2021-09-10', '2021-09-13', '2021-09-14', '2021-09-15', '2021-09-16', '2021-09-17', '2021-09-20', '2021-09-21', '2021-09-22', '2021-09-23', '2021-09-24', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30', '2021-10-01', '2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08', '2021-10-11', '2021-10-12', '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-18', '2021-10-19', '2021-10-20', '2021-10-21', '2021-10-22', '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28', '2021-10-29', '2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05', '2021-11-08', '2021-11-09', '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15', '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19', '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-26', '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03', '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10', '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31'])
    dates = []
    while (date < upToDate):
        if (date.strftime("%Y-%m-%d") in trading_days):
            dates.append(date)
        date += delta
    return dates

In [8]:
stocks = ['SPY', 'FCEL', 'TSLA', 'ONTX', 'AMD', 'AAPL', 'NIO', 'ACB', 'AMZN', 'VBIV', 'BA', 'B', 'ROKU', 'GHSI', 'CHK', 'IBIO', 'UGAZ', 'AMRN', 'BIOC', 'OCGN', 'SPCE', 'SRNE', 'CLVS', 'CODX', 'DIS', 'INO', 'NFLX', 'TBLT', 'VISL', 'MSFT', 'TRNX', 'BABA', 'AIM', 'HTBX', 'AYTU', 'TTOO', 'TVIX', 'GILD', 'JNUG', 'D', 'TOPS', 'USO', 'EROS', 'MARK', 'MVIS', 'GNUS', 'VSTM', 'NVAX']
stocks.sort()
print(stocks)

['AAPL', 'ACB', 'AIM', 'AMD', 'AMRN', 'AMZN', 'AYTU', 'B', 'BA', 'BABA', 'BIOC', 'CHK', 'CLVS', 'CODX', 'D', 'DIS', 'EROS', 'FCEL', 'GHSI', 'GILD', 'GNUS', 'HTBX', 'IBIO', 'INO', 'JNUG', 'MARK', 'MSFT', 'MVIS', 'NFLX', 'NIO', 'NVAX', 'OCGN', 'ONTX', 'ROKU', 'SPCE', 'SPY', 'SRNE', 'TBLT', 'TOPS', 'TRNX', 'TSLA', 'TTOO', 'TVIX', 'UGAZ', 'USO', 'VBIV', 'VISL', 'VSTM']


In [9]:
masked_bull_predictions = np.array(user_predictions, copy=True)
masked_bull_predictions[masked_bull_predictions<0] = 0
count_total_bull = np.count_nonzero(masked_bull_predictions, axis=1) # counts per day per stock
standardize_bull_prediction = masked_bull_predictions / count_total_bull[:,None]
standardize_bull_prediction[np.isnan(standardize_bull_prediction)] = 0

In [10]:
weightings = np.array([1.0, 1.0, 1.0])
user_weighted_bull = np.dot(user_matrice[0], weightings) # 104 x 57000
all_weighted_bull = np.repeat(user_weighted_bull[:,None], 48, axis=1) # 104 x 78 x 57000

# Multiply user weights with user predictions
diagonal_weights_bull = np.einsum('ijk,ikj->ij', all_weighted_bull, standardize_bull_prediction) # 104 x 78

In [11]:
masked_bear_predictions = np.array(user_predictions, copy=True)
masked_bear_predictions[masked_bear_predictions>0] = 0
count_total_bear = np.count_nonzero(masked_bear_predictions, axis=1) # counts per day per stock
standardize_bear_prediction = masked_bear_predictions / count_total_bear[:,None]
standardize_bear_prediction[np.isnan(standardize_bear_prediction)] = 0

In [12]:
user_weighted_bear = np.dot(user_matrice[1], weightings) # 104 x 57000
all_weighted_bear = np.repeat(user_weighted_bear[:,None], 48, axis=1) # 104 x 78 x 57000

# Multiply user weights with user predictions
diagonal_weights_bear = np.einsum('ijk,ikj->ij', all_weighted_bear, standardize_bear_prediction, optimize=True) # 104 x 78

In [13]:
user_stock_weighted_bull = np.dot(user_stock_matrice[0], weightings) # 104 x 57000
user_stock_weighted_bear = np.dot(user_stock_matrice[1], weightings) # 104 x 57000
diagonal_weights_bull_stock = np.einsum('ijk,ikj->ij', user_stock_weighted_bull, standardize_bull_prediction) # 104 x 78
diagonal_weights_bear_stock = np.einsum('ijk,ikj->ij', user_stock_weighted_bear, standardize_bear_prediction) # 104 x 78

In [14]:
diagonal_weights = diagonal_weights_bull + diagonal_weights_bear + diagonal_weights_bull_stock + diagonal_weights_bear_stock

In [19]:
# Find top n stocks to look at
dates = findTradingDays(start_date, end_date)
stocks = np.array(stocks)
diagonal_weights = diagonal_weights[:104]
top_n_stocks = 3
sorted_index = np.argsort(-abs(diagonal_weights),axis=1)
range_i = np.arange(diagonal_weights.shape[0])

# Find top stocks and corresponding close open
top_weights = diagonal_weights[range_i[:,None], sorted_index][:,:top_n_stocks]
top_close_open = close_open_matrice[range_i[:,None], sorted_index][:,:top_n_stocks]
total_weights = np.sum(abs(top_weights), axis=1)[:,None]
relative_weight = top_weights / total_weights
relative_weight[np.isnan(relative_weight)] = 0

stocks_repeated = np.repeat(stocks[None,:], 104, axis=0) 
stocks_repeated = stocks_repeated[range_i[:,None], sorted_index][:,:top_n_stocks]
stock_names = diagonal_weights[range_i[:,None], sorted_index][:,:top_n_stocks]

weighted_returns_perstock = np.multiply(relative_weight, top_close_open)
weighted_returns = np.sum(weighted_returns_perstock, axis=1)

correct = 0
total = 0
for i in range(len(stocks_repeated)):
    res = []
    for j in range(3):
        res.append((stocks_repeated[i][j], round(relative_weight[i][j],2), 
            # round(top_close_open[i][j], 2), round(weighted_returns_perstock[i][j], 2), 
            diagonal_weights[i][sorted_index[i][j]]))
    print((dates[i].month, dates[i].day), round(weighted_returns[i], 2), res)

print(np.sum(weighted_returns))


(1, 9) 1.25 [('VBIV', 0.4, 1.4628927667572007), ('FCEL', 0.3, 1.0774117982443294), ('NIO', 0.3, 1.0753347228626875)]
(1, 10) 3.23 [('ONTX', 0.39, 1.933659878354816), ('GHSI', 0.31, 1.5519909602861537), ('VBIV', 0.3, 1.4864997208304613)]
(1, 13) -1.37 [('VBIV', 0.38, 1.7352593629007065), ('GHSI', 0.37, 1.7046450128268553), ('AMD', 0.25, 1.1572022551487224)]
(1, 14) -0.86 [('VBIV', 0.35, 1.9854186138687238), ('GHSI', 0.35, 1.980774514390144), ('IBIO', 0.3, 1.6620429036525588)]
(1, 15) -1.23 [('GHSI', 0.4, 2.3246654769890034), ('VBIV', 0.32, 1.870348644337032), ('BIOC', 0.28, 1.6630721598521738)]
(1, 16) 0.02 [('VBIV', 0.43, 2.2619245631205853), ('AMRN', 0.3, 1.5833185242323409), ('AMD', 0.27, 1.3886316726347567)]
(1, 17) 4.34 [('IBIO', 0.37, 1.9311085825625764), ('GHSI', 0.37, 1.9015446695480733), ('AMRN', 0.26, 1.3176197200670672)]
(1, 21) 1.61 [('VBIV', 0.44, 2.2887565232779767), ('AMD', 0.28, 1.449609762509529), ('SPCE', 0.28, 1.4444269481702823)]
(1, 22) -1.24 [('VBIV', 0.4, 1.832998

In [18]:
import statistics
import datetime
start_date = datetime.datetime(2020, 1, 9)
end_date = datetime.datetime(2020, 6, 9)
dates = findTradingDays(start_date, end_date)

stock_mean_std = {}
for i in range(104):
    weights = diagonal_weights[i]
    top_3_index = sorted(range(len(weights)), key=lambda x: abs(weights[x]))[-20:]
    res = []
    for j in top_3_index:
        symbol = stocks[j]
        weight = round(weights[j], 2)
        res.append([symbol, weight])
        if (symbol not in stock_mean_std):
            stock_mean_std[symbol] = []
        stock_mean_std[symbol].append(weight)

mean_std = {}
for symbol in stock_mean_std:
    mean_std[symbol] = {
        'std': statistics.stdev(stock_mean_std[symbol]),
        'avg': statistics.mean(stock_mean_std[symbol])
    }

# print(statistics.stdev([0,0]))
total = 0
count = 0
correct = 0
for i in range(104):
    weights = diagonal_weights[i]
    new_weight = []
    for j in range(len(weights)):
        if (weights[j] == 0):
            continue
        symbol = stocks[j]
        stdDev = (weights[j] - mean_std[symbol]['avg']) / mean_std[symbol]['std']
        new_weight.append(stdDev)

    top_3_index = sorted(range(len(new_weight)), key=lambda x: abs(new_weight[x]))[-3:]
    res = []
    total_weight = 0
    for j in top_3_index:
        symbol = stocks[j]
        index_symbol = list(stocks).index(symbol)
        weight = round(new_weight[j], 2)
        # print(symbol, i, index_symbol)
        percent_change = round(close_open_matrice[i][index_symbol], 2)
        res.append([symbol, weight, percent_change])
        total_weight += abs(weight)
    
    percentToday = 0
    for r in res:
        # print(r)
        p = r[1] * r[2] / total_weight
        if (p > 0):
            correct += 1
        count += 1
        percentToday += p
    print(dates[i], round(percentToday, 2), res)
    total += percentToday
print(total, correct, count)

2020-01-09 00:00:00 0.46 [['EROS', 1.03, 0.54], ['CHK', 1.64, 1.25], ['DIS', -1.76, 0.32]]
2020-01-10 00:00:00 0.23 [['D', 1.38, 0.01], ['CLVS', 1.7, 1.31], ['EROS', 1.99, -0.55]]
2020-01-13 00:00:00 -0.53 [['EROS', 1.51, -2.48], ['B', -1.57, -0.25], ['FCEL', 1.7, 0.49]]
2020-01-14 00:00:00 -3.35 [['CODX', 1.62, -6.31], ['ACB', -1.66, 3.35], ['FCEL', 2.01, -0.96]]
2020-01-15 00:00:00 -0.51 [['AMD', -1.32, 1.28], ['B', 1.69, 0.26], ['AIM', 1.78, -0.68]]
2020-01-16 00:00:00 -0.38 [['ACB', -1.64, 0.0], ['D', -1.65, 0.27], ['B', -1.91, 0.79]]
2020-01-17 00:00:00 0.16 [['DIS', 1.44, -0.06], ['CHK', 2.02, -0.14], ['B', -2.35, -0.56]]
2020-01-21 00:00:00 -0.09 [['BABA', -1.54, 1.09], ['AYTU', -1.6, 0.0], ['DIS', 1.99, 0.62]]
2020-01-22 00:00:00 -0.41 [['CHK', 1.56, -1.56], ['DIS', 1.78, -0.28], ['AMD', -2.7, -0.17]]
2020-01-23 00:00:00 -0.86 [['BIOC', 1.47, -1.97], ['ACB', -1.51, 1.45], ['CLVS', 1.56, 0.76]]
2020-01-24 00:00:00 0.86 [['D', 1.03, -0.06], ['EROS', 1.11, -2.84], ['GHSI', -1.15, 

In [198]:

user_matrice = np.load('user_matrice_filtered.npy') # 104 days x 57000 users x 3 weightings
last = np.repeat([user_matrice[-1]],repeats=8 ,axis=0)
user_matrice = np.vstack([user_matrice, last])

In [46]:
last = np.repeat([user_stock_matrice[-1]],repeats=8 ,axis=0)
user_stock_matrice = np.vstack([user_stock_matrice, last])
user_stock_matrice.shape

(112, 78, 57394, 3)

In [4]:
import numpy as np
user_matrice = np.load('user_matrice.npy')

# set min tweets at 20
user_matrice[:,:,:,0][user_matrice[:,:,:,0] <= 20] = 0

In [14]:
# Mask all 0 values
masked_user_matrice = np.ma.masked_equal(user_matrice, 0)

# Find average/std for tweet number and standardize based on that
# summed = masked_user_matrice[:,:,:,0].sum(axis=1)
non_zero_count = np.count_nonzero(masked_user_matrice[:,:,:,0], axis=1)

In [80]:

user_matrice = np.load('user_matrice.npy')

bull_matrice = user_matrice[0]
bull_matrice[:,:,0][bull_matrice[:,:,0] <= 20] = 0
bull_matrice = np.ma.masked_equal(bull_matrice, 0)
non_zero_count = np.count_nonzero(bull_matrice[:,:,0], axis=1)
summed = bull_matrice[:,:,0].sum(axis=1)
mean = summed / non_zero_count
std = np.std(bull_matrice[:,:,0], axis=1)

bull_matrice[:,:,0] = (bull_matrice[:,:,0] - mean[:,None]) / std[:,None]
bull_matrice[:,:,0] = np.add(bull_matrice[:,:,0], 2)
bull_matrice[:,:,0][bull_matrice[:,:,0] <= 0] = 0
bull_matrice[:,:,0][bull_matrice[:,:,0] > 4] = 4
bull_matrice[:,:,0] = np.divide(bull_matrice[:,:,0], 4)
bull_matrice[bull_matrice.mask] = 0
user_matrice[0] = bull_matrice


bear_matrice = user_matrice[1]
bear_matrice[:,:,0][bear_matrice[:,:,0] <= 10] = 0
bear_matrice = np.ma.masked_equal(bear_matrice, 0)
non_zero_count = np.count_nonzero(bear_matrice[:,:,0], axis=1)
summed = bear_matrice[:,:,0].sum(axis=1)
mean = summed / non_zero_count
std = np.std(bear_matrice[:,:,0], axis=1)

bear_matrice[:,:,0] = (bear_matrice[:,:,0] - mean[:,None]) / std[:,None]
bear_matrice[:,:,0] = np.add(bear_matrice[:,:,0], 2)
bear_matrice[:,:,0][bear_matrice[:,:,0] <= 0] = 0
bear_matrice[:,:,0][bear_matrice[:,:,0] > 4] = 4
bear_matrice[:,:,0] = np.divide(bear_matrice[:,:,0], 4)
bear_matrice[bear_matrice.mask] = 0
user_matrice[1] = bear_matrice


print(user_matrice[1,100,:20])
min_accuracy = 0.5
user_matrice[:,:,:,1][user_matrice[:,:,:,1] <= min_accuracy] = 0
    

min_return = -50
max_return = 250
diff = max_return - min_return
user_matrice[:,:,:,2][user_matrice[:,:,:,2] <= min_return] = min_return
user_matrice[:,:,:,2][user_matrice[:,:,:,2] > max_return] = max_return
user_matrice[:,:,:,2] = np.add(user_matrice[:,:,:,2], -min_return)
user_matrice[:,:,:,2] = np.divide(user_matrice[:,:,:,2], diff)

print(user_matrice[1,100,:20])




[[  0.           0.           0.        ]
 [  0.           1.          12.60702687]
 [  0.           0.375        0.47033323]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.66666667  18.60328746]
 [  0.           0.           0.        ]
 [  0.           0.57142857  13.21421545]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.47962779   0.66666667  60.54015763]
 [  0.           0.           0.        ]
 [  0.           0.25       -12.57380264]
 [  0.           0.66666667  14.8410678 ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.71428571   2.1505891 ]]
[[0.         0.         0.16666667]
 [0.         1.         0.20869009]
 [0.         0.         0.16823444]
 [0.         0.         0.16666667]
 [0.         0.